# **Exercise**
---    

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

X = pd.read_csv("../datafile/train.csv", index_col='Id')
X_test = pd.read_csv("../datafile/train.csv", index_col='Id')

X.dropna(axis=0,subset=['SalePrice'],inplace=True)
y = X.SalePrice    
X.drop(['SalePrice'],axis=1,inplace=True)

cols_with_missing = [col for col in X.columns if X[col].isnull().any()]

X.drop(cols_with_missing, axis=1,inplace=True)
X_test.drop(cols_with_missing,axis=1,inplace=True)


X_train, X_valid, y_train, y_valid = train_test_split(X,y,train_size=0.8,test_size=0.2,random_state=0)
X_train.head()

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
619,20,RL,11694,Pave,Reg,Lvl,AllPub,Inside,Gtl,NridgHt,...,108,0,0,260,0,0,7,2007,New,Partial
871,20,RL,6600,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,...,0,0,0,0,0,0,8,2009,WD,Normal
93,30,RL,13360,Pave,IR1,HLS,AllPub,Inside,Gtl,Crawfor,...,0,44,0,0,0,0,8,2009,WD,Normal
818,20,RL,13265,Pave,IR1,Lvl,AllPub,CulDSac,Gtl,Mitchel,...,59,0,0,0,0,0,7,2008,WD,Normal
303,20,RL,13704,Pave,IR1,Lvl,AllPub,Corner,Gtl,CollgCr,...,81,0,0,0,0,0,1,2006,WD,Normal


In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def score_dataset(X_train,X_valid,y_train,y_valid):
    model = RandomForestRegressor(n_estimators=100,random_state=0)
    model.fit(X_train,y_train)
    preds = model.predict(X_valid) 
    return mean_absolute_error(y_valid,preds)

### **문제1**
Categorical Variables drop하기

### **답**

In [3]:
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

17837.82570776256


### **유의할 점**

In [4]:
print(X_train['Condition2'].unique())
print(X_valid['Condition2'].unique())

['Norm' 'PosA' 'Feedr' 'PosN' 'Artery' 'RRAe']
['Norm' 'RRAn' 'RRNn' 'Artery' 'Feedr' 'PosN']


위 두개의 값이 다르다. 이럴 경우 test data에 train되지 않은 데이터가 있으므로 오류가 발생 한다.

In [20]:
object_cols = [col for col in X_train.columns if X_train[col].dtype == 'object']

good_label_cols = [col for col in object_cols if set(X_valid[col]).issubset(set(X_train[col]))]

bad_label_cols = list(set(object_cols) -set(good_label_cols))

print(good_label_cols)
print("===============================")
print(bad_label_cols)

['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'BldgType', 'HouseStyle', 'RoofStyle', 'Exterior1st', 'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation', 'Heating', 'HeatingQC', 'CentralAir', 'KitchenQual', 'PavedDrive', 'SaleType', 'SaleCondition']
['Functional', 'RoofMatl', 'Condition2']


### **문제2**
good_label에 대해서만 ordinal encode

### **답**

In [23]:
from sklearn.preprocessing import OrdinalEncoder

label_X_train = X_train.drop(bad_label_cols, axis=1)   
label_X_valid = X_valid.drop(bad_label_cols, axis=1)

oe = OrdinalEncoder()

label_X_train[good_label_cols] = oe.fit_transform(label_X_train[good_label_cols])
label_X_valid[good_label_cols] = oe.transform(label_X_valid[good_label_cols])

print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

17098.01649543379


### **유의할 점**
one hot encoding을 위해서는 집합의 크기가 10 미만인 값에만 적용하는 것이 좋다.

In [27]:
object_unique = list(map(lambda col: X_train[col].nunique(), object_cols))
d = dict(zip(object_cols, object_unique))

sorted(d.items(), key=lambda x:x[1])

[('Street', 2),
 ('Utilities', 2),
 ('CentralAir', 2),
 ('LandSlope', 3),
 ('PavedDrive', 3),
 ('LotShape', 4),
 ('LandContour', 4),
 ('ExterQual', 4),
 ('KitchenQual', 4),
 ('MSZoning', 5),
 ('LotConfig', 5),
 ('BldgType', 5),
 ('ExterCond', 5),
 ('HeatingQC', 5),
 ('Condition2', 6),
 ('RoofStyle', 6),
 ('Foundation', 6),
 ('Heating', 6),
 ('Functional', 6),
 ('SaleCondition', 6),
 ('RoofMatl', 7),
 ('HouseStyle', 8),
 ('Condition1', 9),
 ('SaleType', 9),
 ('Exterior1st', 15),
 ('Exterior2nd', 16),
 ('Neighborhood', 25)]

In [24]:
low_cardinality_cols = [col for col in object_cols if X_train[col].nunique() < 10]

high_cardinality_cols = list(set(object_cols)-set(low_cardinality_cols))

print(low_cardinality_cols)
print("========================")
print(high_cardinality_cols)

['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'ExterQual', 'ExterCond', 'Foundation', 'Heating', 'HeatingQC', 'CentralAir', 'KitchenQual', 'Functional', 'PavedDrive', 'SaleType', 'SaleCondition']
['Exterior1st', 'Neighborhood', 'Exterior2nd']


In [30]:
from sklearn.preprocessing import OneHotEncoder

OH = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH.fit_transform(X_train[low_cardinality_cols]))
OH_cols_valid = pd.DataFrame(OH.transform(X_valid[low_cardinality_cols]))

OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

C:\Users\Administrator\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


17525.345719178084


C:\Users\Administrator\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
